In [1]:
from modeloMultiplo import MRLM
import pandas as pd

In [2]:
df = pd.read_excel("dados_projeto.xlsx")
df

,Investimento_marketing,alcance_midias,cliques,taxa_conversao,taxa_ocupacao_hoteleira,estacao,tipo_destino,orcamento_publico,preco_medio_pacote,reputacao_online,eventos_anuais
0,319.24,27.89,23.19,8.04,64.7,Media,Praia,85.8,3.84,3.58,2.0
1,274.77,21.66,16.70,7.80,70.5,Alta,NaN,42.1,2.50,5.00,1.0
2,195.16,19.56,9.79,8.35,69.9,Media,Urbano,52.6,1.92,1.84,3.0
3,310.48,31.94,26.59,6.54,57.3,Alta,Campo,53.1,3.93,4.97,7.0
4,241.81,28.98,22.75,8.30,61.4,Alta,Urbano,43.6,5.85,4.26,1.0
...,...,...,...,...,...,...,...,...,...,...,...
895,275.19,36.82,31.79,6.98,64.0,Media,Praia,63.7,4.08,4.29,1.0
896,282.17,22.13,16.75,2.07,71.5,Media,Urbano,93.6,4.09,2.50,2.0
897,230.21,19.35,11.86,7.94,70.2,Media,Urbano,33.2,2.82,5.00,3.0
898,207.22,17.57,17.72,5.13,66.4,Media,Urbano,58.4,1.48,4.53,4.0


In [3]:
# estacao_ref = ["", "Baixa", "Media", "Alta"]
# estacao_dict = {}
# for i, ref in enumerate(estacao_ref):
#     estacao_dict[ref] = i
# df["estacao"] = df["estacao"].replace(estacao_dict)

# tipo_destino_ref = ['', 'Praia', 'Urbano', 'Campo', 'Cultural']
# tipo_destino_dict = {}
# for i, ref in enumerate(tipo_destino_ref):
#     tipo_destino_dict[ref] = i
# df["tipo_destino"] = df["tipo_destino"].replace(tipo_destino_dict)

In [4]:
dummies = ["estacao", "tipo_destino"]
df = pd.get_dummies(df, columns=dummies, drop_first=True)
bool_cols = df.select_dtypes(include='bool').columns
df[bool_cols] = df[bool_cols].astype(int)

In [5]:
df

,Investimento_marketing,alcance_midias,cliques,taxa_conversao,taxa_ocupacao_hoteleira,orcamento_publico,preco_medio_pacote,reputacao_online,eventos_anuais,estacao_Baixa,estacao_Media,tipo_destino_Cultural,tipo_destino_Praia,tipo_destino_Urbano
0,319.24,27.89,23.19,8.04,64.7,85.8,3.84,3.58,2.0,0,1,0,1,0
1,274.77,21.66,16.70,7.80,70.5,42.1,2.50,5.00,1.0,0,0,0,0,0
2,195.16,19.56,9.79,8.35,69.9,52.6,1.92,1.84,3.0,0,1,0,0,1
3,310.48,31.94,26.59,6.54,57.3,53.1,3.93,4.97,7.0,0,0,0,0,0
4,241.81,28.98,22.75,8.30,61.4,43.6,5.85,4.26,1.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,275.19,36.82,31.79,6.98,64.0,63.7,4.08,4.29,1.0,0,1,0,1,0
896,282.17,22.13,16.75,2.07,71.5,93.6,4.09,2.50,2.0,0,1,0,0,1
897,230.21,19.35,11.86,7.94,70.2,33.2,2.82,5.00,3.0,0,1,0,0,1
898,207.22,17.57,17.72,5.13,66.4,58.4,1.48,4.53,4.0,0,1,0,0,1


In [6]:
y_label = "Investimento_marketing"

y = df[y_label]
X = df.drop(columns=y_label)

In [7]:
modelo = MRLM(X, y, X.columns.to_list(), y.name, dummies)

X = df.drop(columns=["cliques", "taxa_ocupacao_hoteleira", "eventos_anuais", "Investimento_marketing"])
modelo2 = MRLM(X, y, X.columns.to_list(), y.name)

In [ ]:
modelo_melhorado = modelo.selecionar_melhor_combinacao_de_variaveis(criterio="AIC")

1023 / 1023     

In [9]:
modelo.sumario_sm()

                              OLS Regression Results                              
Dep. Variable:     Investimento_marketing   R-squared:                       0.311
Model:                                OLS   Adj. R-squared:                  0.299
Method:                     Least Squares   F-statistic:                     26.74
Date:                    Tue, 23 Sep 2025   Prob (F-statistic):           5.20e-54
Time:                            17:55:46   Log-Likelihood:                -3999.0
No. Observations:                     784   AIC:                             8026.
Df Residuals:                         770   BIC:                             8091.
Df Model:                              13                                         
Covariance Type:                nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------

In [10]:
modelo_melhorado.sumario_sm()

                              OLS Regression Results                              
Dep. Variable:     Investimento_marketing   R-squared:                       0.300
Model:                                OLS   Adj. R-squared:                  0.291
Method:                     Least Squares   F-statistic:                     33.07
Date:                    Tue, 23 Sep 2025   Prob (F-statistic):           1.32e-53
Time:                            17:55:46   Log-Likelihood:                -4005.4
No. Observations:                     784   AIC:                             8033.
Df Residuals:                         773   BIC:                             8084.
Df Model:                              10                                         
Covariance Type:                nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------

In [11]:
# modelo.plot_correlacao_em_pares(show_model=True)

In [12]:
# modelo.sumario_em_pares()

In [13]:
# modelo.residual_plots()